In [1]:
# Importando los módulos que necesitaremos
import numpy as np

import pandas as pd
from pandas.plotting import table

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import seaborn as sns

from datetime import datetime

import os

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [2]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunas_covid.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210302,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210302,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
2,20210302,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210302,746ad64c513b5d53b987ed48a6b45f77,PERSONAL DE SALUD,54.0,FEMENINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210302,80f0a41dd63af3b96d4a80bfab2eaa61,PERSONAL DE SALUD,35.0,FEMENINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO


In [3]:
# Obtengo el conteo del número de valores en cada columnas
df.count()

FECHA_CORTE         298832
UUID                298832
GRUPO_RIESGO        298832
EDAD                298659
SEXO                297682
FECHA_VACUNACION    298832
DOSIS               298832
FABRICANTE          298832
DIRESA              298832
DEPARTAMENTO        298832
PROVINCIA           298832
DISTRITO            298832
dtype: int64

In [4]:
# Verificamos que las columnas DEPARTAMENTO, GRUPO_RIESGO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['GRUPO_RIESGO'].unique())
print(df['SEXO'].unique())

['LAMBAYEQUE' 'LIMA' 'AMAZONAS' 'LA LIBERTAD' 'CAJAMARCA' 'AYACUCHO'
 'APURIMAC' 'SAN MARTIN' 'PIURA' 'HUANUCO' 'LORETO' 'CALLAO' 'CUSCO' 'ICA'
 'UCAYALI' 'HUANCAVELICA' 'AREQUIPA' 'JUNIN' 'MADRE DE DIOS' 'PUNO'
 'PASCO' 'ANCASH' 'MOQUEGUA' 'TACNA' 'TUMBES']
['PERSONAL DE SALUD' 'TRABAJADOR Ó PERSONAL DE LIMPIEZA'
 'PERSONAL DE SEGURIDAD' 'PERSONAL MILITAR Ó FF AA'
 'POLICIA NACIONAL DEL PERU' 'ESTUDIANTES DE CIENCIAS DE LA SALUD'
 'BRIGADISTAS' 'CRUZ ROJA']
['MASCULINO' 'FEMENINO' nan]


In [5]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [6]:
# Abreviando los grupos de riesgo
df['GRUPO_RIESGO'].replace('PERSONAL DE SALUD','P. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('TRABAJADOR Ó PERSONAL DE LIMPIEZA','T. LIMP.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL DE SEGURIDAD','P. SEG.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL MILITAR Ó FF AA','FF.AA.', inplace=True)
df['GRUPO_RIESGO'].replace('POLICIA NACIONAL DEL PERU','P.N.P.', inplace=True)
df['GRUPO_RIESGO'].replace('ESTUDIANTES DE CIENCIAS DE LA SALUD','E. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('BRIGADISTAS','BRIG.', inplace=True)

# Genero Serie de grupos de riesgo
grupos_riesgo = df['GRUPO_RIESGO'].unique()
grupos_riesgo_series = pd.Series(np.zeros_like(grupos_riesgo),grupos_riesgo)

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb_age = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y GRUPO_RIESGO para el DataFrame
gb_gr = df[["DEPARTAMENTO","SEXO",'GRUPO_RIESGO']].groupby(['DEPARTAMENTO', 'SEXO', 'GRUPO_RIESGO'])

In [7]:
def get_data_to_plot(department=None):
    if(department is None):
        return df[['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]
    else:
        return df[df['DEPARTAMENTO'] == department][['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]

## Visualización de Datos

In [8]:
# Visualizo los datos agrupados por EDAD via DataFrame
gb_age.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0        10       644      1109       642   
              MASCULINO        0         7       333       618       439   
ANCASH        FEMENINO         0         1      1099      1927      1433   
              MASCULINO        0         6       435       935       746   
APURIMAC      FEMENINO         0         3       834      1458      1104   
              MASCULINO        0         4       309       722       691   
AREQUIPA      FEMENINO         0         4      1634      3076      2351   
              MASCULINO        0         4       565      1205      1026   
AYACUCHO      FEMENINO         0         4       894      1619      1214   
              MASCULINO        0         5       320       789       779   
CAJAMARCA     FEMENINO         0         2      1451      2310      1470   
              MASCULINO        0        10       639      1231       858   
CALLAO        FEMENINO         0        17      1328      3430      2872   
              MASCULINO        0        22       899      1686      1544   
CUSCO         FEMENINO         0         9      1034      2577      1708   
              MASCULINO        0         9       429      1075       946   
HUANCAVELICA  FEMENINO         0         6       873      1253       712   
              MASCULINO        0         4       330       584       456   
HUANUCO       FEMENINO         0         4      1018      1729      1107   
              MASCULINO        0        11       391       929       719   
ICA           FEMENINO         0         3      1210      1684      1200   
              MASCULINO        0         5       412       743       605   
JUNIN         FEMENINO         0         4      1890      2688      1610   
              MASCULINO        0         9       563      1126       771   
LA LIBERTAD   FEMENINO         0         4      1734      3198      2144   
              MASCULINO        0        10       804      1464      1204   
LAMBAYEQUE    FEMENINO         0         1      1147      2025      1544   
              MASCULINO        0         1       446       906       803   
LIMA          FEMENINO         0        55      9338     22430     18017   
              MASCULINO        0        82      4781     10807      9180   
LORETO        FEMENINO         0         4       571      1301       896   
              MASCULINO        0         7       422       881       606   
MADRE DE DIOS FEMENINO         0         4       179       365       266   
              MASCULINO        0         1        75       179       129   
MOQUEGUA      FEMENINO         0         3       429       743       645   
              MASCULINO        0         5       139       342       287   
PASCO         FEMENINO         0         1       430       622       461   
              MASCULINO        0         3       162       309       295   
PIURA         FEMENINO         0         9      1525      2778      2423   
              MASCULINO        0        13       748      1414      1308   
PUNO          FEMENINO         0         3       905      1745      1132   
              MASCULINO        0         0       293       747       639   
SAN MARTIN    FEMENINO         0         9       936      1669       836   
              MASCULINO        0        11       529      1019       581   
TACNA         FEMENINO         0         4       455       818       757   
              MASCULINO        0         7       238       370       359   
TUMBES        FEMENINO         0         0       337       551       458   
              MASCULINO        0         1       181       298       269   
UCAYALI       FEMENINO         0         3       593       927       668   
              MASCULINO        0         6       366       549       441   

EDAD                     [50, 60)  [60, 70)  [7

In [9]:
# Visualizo los datos agrupados por GRUPO_RIESGO via DataFrame
gb_gr.size().unstack()

GRUPO_RIESGO             BRIG.  CRUZ ROJA  E. SALUD  FF.AA.  P. SALUD  \
DEPARTAMENTO  SEXO                                                      
AMAZONAS      FEMENINO     NaN        NaN       NaN     9.0    2724.0   
              MASCULINO    NaN        NaN       NaN    16.0    1659.0   
ANCASH        FEMENINO     NaN        NaN      37.0     NaN    5941.0   
              MASCULINO    NaN        NaN      26.0     5.0    2944.0   
APURIMAC      FEMENINO     NaN        NaN       NaN     NaN    4017.0   
              MASCULINO    NaN        NaN       NaN     NaN    2280.0   
AREQUIPA      FEMENINO     NaN        NaN       NaN     NaN   10384.0   
              MASCULINO    NaN        NaN       NaN     NaN    4289.0   
AYACUCHO      FEMENINO     NaN        NaN       4.0     NaN    4539.0   
              MASCULINO    NaN        NaN       3.0     NaN    2588.0   
CAJAMARCA     FEMENINO     NaN        NaN      13.0     NaN    6022.0   
              MASCULINO    NaN        NaN      12.0     NaN    3280.0   
CALLAO        FEMENINO     NaN        NaN       NaN    48.0   10629.0   
              MASCULINO    NaN        NaN       NaN    19.0    5751.0   
CUSCO         FEMENINO     NaN        NaN       NaN     NaN    7007.0   
              MASCULINO    NaN        NaN       NaN     3.0    3405.0   
HUANCAVELICA  FEMENINO     NaN        NaN       NaN     NaN    3262.0   
              MASCULINO    NaN        NaN       NaN     3.0    1699.0   
HUANUCO       FEMENINO     NaN        NaN       1.0     NaN    4635.0   
              MASCULINO    NaN        NaN       3.0     NaN    2565.0   
ICA           FEMENINO     NaN        NaN      53.0     1.0    5521.0   
              MASCULINO    NaN        NaN      22.0     NaN    2551.0   
JUNIN         FEMENINO     NaN        NaN      85.0     8.0    7539.0   
              MASCULINO    NaN        NaN      22.0    21.0    3168.0   
LA LIBERTAD   FEMENINO     NaN        NaN      12.0     NaN    9088.0   
              MASCULINO    NaN        NaN       5.0     NaN    4594.0   
LAMBAYEQUE    FEMENINO     NaN        NaN       NaN    15.0    6187.0   
              MASCULINO    NaN        NaN       NaN     4.0    3013.0   
LIMA          FEMENINO     1.0        NaN       9.0    19.0   71053.0   
              MASCULINO    1.0        NaN       3.0    15.0   36134.0   
LORETO        FEMENINO     NaN        NaN       3.0     1.0    3436.0   
              MASCULINO    NaN        NaN       1.0     1.0    2463.0   
MADRE DE DIOS FEMENINO     NaN        NaN       NaN     NaN    1012.0   
              MASCULINO    NaN        NaN       NaN     NaN     524.0   
MOQUEGUA      FEMENINO     NaN        NaN       NaN     NaN    2321.0   
              MASCULINO    NaN        NaN       NaN     NaN    1124.0   
PASCO         FEMENINO     NaN        NaN       2.0     2.0    1856.0   
              MASCULINO    NaN        NaN       NaN     2.0     995.0   
PIURA         FEMENINO     NaN        NaN       7.0     5.0    8489.0   
              MASCULINO    1.0        NaN      14.0     2.0    4860.0   
PUNO          FEMENINO     NaN        NaN       1.0     NaN    4954.0   
              MASCULINO    NaN        NaN       NaN     NaN    2568.0   
SAN MARTIN    FEMENINO     NaN        NaN       NaN     NaN    4092.0   
              MASCULINO    NaN        NaN       NaN     NaN    2680.0   
TACNA         FEMENINO     NaN        NaN       NaN     NaN    2899.0   
              MASCULINO    NaN        NaN       NaN     NaN    1542.0   
TUMBES        FEMENINO     NaN        1.0       NaN     NaN    1582.0   
              MASCULINO    NaN        NaN       NaN     NaN     904.0   
UCAYALI       FEMENINO     NaN        NaN       NaN     NaN    2672.0   
              MASCULINO    NaN        NaN       NaN     2.0    1769.0   

GRUPO_RIESGO             P. SEG.  P.N.P.  T. LIMP.  
DEPARTAMENTO  SEXO                                  
AMAZONAS      FEMENINO       6.0    15.0      65.0  
              MASCULINO     33.0     7.0    

In [10]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(area, data, date_range, show_plot=False, save_plot=True):

    # Creamos la figura, los ejes y agregamos la atribución
    plt.clf()

    fig, axs = plt.subplots(2,2, figsize=(20,20))
    plt.figtext(0.1,0.95, area + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.925, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{datetime.now():%Y-%m-%d}', ha='right')

    # 1. Graficamos distribución por GRUPO ETAREO
    xlim_max = 0

    ## 1.1. Graficamos el primer histograma
    N, bins, patches = axs[0][0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = N.max() * 1.1
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0][0].set_yticklabels([])
    axs[0][0].yaxis.set_tick_params(width=0)
    axs[0][0].set_yticks(yticks, minor=True)
    axs[0][0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0][0].set_ylim(0,90)
    axs[0][0].set_xlim(0,xlim_max)
    axs[0][0].set_ylabel('GRUPO ETÁREO (AÑOS)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][0].text(N[i], yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 1.2. Graficamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[0][1].hist([males, females], BINS, orientation = 'horizontal')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[0][1].set_yticklabels([])
    axs[0][1].yaxis.set_tick_params(width=0)
    axs[0][1].set_yticks(yticks, minor=True)
    axs[0][1].set_ylim(0,90)
    axs[0][1].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][1].text(N[0][i], yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[0][1].text(N[1][i], yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[0][1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    # 2. Graficamos distribución por GRUPO RIESGO
    xlim_max = 0

    ## 2.1. Graficamos el primer gráfico de barras 
    group_counts = data['GRUPO_RIESGO'].value_counts().add(grupos_riesgo_series,fill_value=0)
    group_counts.sort_index(inplace=True,ascending=False)
    pos = np.array(range(len(group_counts)))
    patches = axs[1][0].barh(pos, group_counts.values, align='center')
    
    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = group_counts.values.max() * 1.1
    fracs = group_counts.values / group_counts.values.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[1][0].set_yticks(pos)
    axs[1][0].set_yticklabels(list(group_counts.index.values))
    axs[1][0].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][0].text(group_counts.values[i], pos[i]-0.1, str(round(group_counts[i]*100/len(data),2)) + '%')

    ## 2.2. Graficamos el segundo gráfico de barras 
    males = data[data['SEXO'] == 'MASCULINO']
    males_group_counts = males["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    males_group_counts.sort_index(inplace=True,ascending=False)

    females = data[data['SEXO'] == 'FEMENINO']
    females_group_counts = females["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    females_group_counts.sort_index(inplace=True,ascending=False)

    pos = np.array(range(len(males_group_counts)))

    axs[1][1].barh(pos, males_group_counts.values, 0.35, align='center')
    axs[1][1].barh(pos+0.35, females_group_counts.values, 0.35, align='center')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[1][1].set_yticks(pos)
    axs[1][1].set_yticklabels([])
    axs[1][1].set_xlim(0,xlim_max)
    
    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][1].text(males_group_counts.values[i], pos[i]-0.1, str(round(males_group_counts[i]*100/len(data),2)) + '%')
        axs[1][1].text(females_group_counts.values[i], pos[i]+0.3, str(round(females_group_counts[i]*100/len(data),2)) + '%')

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)

    if save_plot:
        filename = 'dist/{}_{}.png'.format(f'{datetime.now():%Y%m%d}', area.replace(' ', '_'))
        plt.savefig("../" + filename, bbox_inches='tight')
        with open("../dist/images.txt", "a", encoding='utf-8') as f:
            f.write('![alt text]({} "{}")\n'.format(filename, area))

    if show_plot:
        plt.show()

In [11]:
# Eliminando archivo generado para el Markdown de las imágenes
if os.path.exists("../dist/images.txt"):
    os.remove("../dist/images.txt")

In [12]:
# Perú
data = get_data_to_plot()
dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
generate_plot('PERÚ', data, (dates.min(), dates.max()), False, True)

In [13]:
# Deparmanetos
departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_to_plot(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()), False, True)